# Paginate through widgets

This notebook demonstrates how to use the `next` parameter with the [`getwidgets`](https://developers.mural.co/public/reference/getmuralwidgets) MURAL API endpoint.

The sample mural for this notebook is here: [Sample mural](https://github.com/spackows/MURAL-API-Samples/blob/main/murals/sample-04_sentiment.json)

You can create that mural using the notebook here: [Mural creating notebook](https://github.com/spackows/MURAL-API-Samples/blob/main/notebooks/sample-01_create-a-mural.ipynb)

The sample mural for this notebook looks like the following image:

<img src="https://raw.githubusercontent.com/spackows/MURAL-API-Samples/main/images/sample-04_sentiment.png" width="50%" title="Image of a mural" />

## Step 1: Collect the mural ID

You can find the mural ID in the url of a mural.

Mural urls look something like this:

```
https://app.mural.co/t/<workspace>/m/<workspace>/<id>/...
```

What you need to pass to the MURAL API is just after the `/m/`: the `<workspace>` and the `<id>`.  And you need to join them with a period.

For example, if you have a mural with this url:

```
https://app.mural.co/t/teamideas1234/m/teamideas1234/1234567890123/...
```

Then, the mural ID is: `teamideas1234.1234567890123`

In [3]:
g_mural_id = ""

## Step 2: Collect your OAuth token

In [2]:
g_auth_token = ""

## Step 3: Read the widgets from the mural

In this sample, we'll use the `limit` parameter to cause the `getwidgets` endpoint to return only 3 widgets in each call.

There are more than a dozen widgets (sticky notes) in the sample mural.  So, we'll need to call the endpoint multiple times with the `next` parameter to get all the widgets.

In [22]:
import requests
import json

def listWidgets( mural_id, auth_token, next_token ):
    # https://developers.mural.co/public/reference/getmuralwidgets
    url = "https://app.mural.co/api/public/v1/murals/" + mural_id + "/widgets?limit=3"
    if next_token:
        url += "&next=" + next_token
    headers = { "Accept": "application/json", "Authorization": "Bearer " + auth_token }
    response = requests.request( "GET", url, headers = headers )
    response_json = json.loads( response.text )
    msg = ""
    if "code" in response_json:
        msg += response_json["code"] + " "
    if "message" in response_json:
        msg += response_json["message"]
    if msg != "":
        print( msg )
        return { "value" : [] }
    if "value" not in response_json:
        print( "No value returned" )
        return { "value" : [] }
    return response_json

def printInfo( response_json, counter ):
    print( "[ " + str( counter ) + " ]" )
    widgets_arr = response_json["value"]
    for widget in widgets_arr:
        print( widget["text"] )
    if( "next" in response_json ):
        print( "\n'next' : " + response_json["next"][0:10] + "..." )
    else:
        print( "\n'next' : None" )
    print( "" )

def listAllWidgets( mural_id, auth_token ):
    all_results = []
    response_json = listWidgets( mural_id, auth_token, None )
    counter = 1                                # For demo purposes only
    printInfo( response_json, counter )        # For demo purposes only
    counter += 1                               # For demo purposes only
    all_results += response_json["value"]
    while "next" in response_json:
        response_json = listWidgets( mural_id, auth_token, response_json["next"] )
        printInfo( response_json, counter )    # For demo purposes only
        counter += 1                           # For demo purposes only
        all_results += response_json["value"]
    return all_results

In [23]:
widgets_arr = listAllWidgets( g_mural_id, g_auth_token )
print( "\nTotal number of widgets: " + str( len( widgets_arr ) ) )

[ 1 ]
I love winter!
I don't do well in the heat
Fall is colourful

'next' : ZmVmZjAzN2...

[ 2 ]
Spring makes me feel optimistic
I hate the cold
Fall leaves are beautiful

'next' : NmZjNTkyNm...

[ 3 ]
Nothing is as soothing as drinking hot chocolate after being out in the cold
Swimming in the summer makes me so happy
I feel sad watching everything die off in the fall

'next' : NTgxMTMwMW...

[ 4 ]
summer is for relaxing
spring and fall are shorter
seasons bring a change of pace

'next' : MmM5NDMzYm...

[ 5 ]
We go south for the winter every year

'next' : None


Total number of widgets: 13
